## Testing MCP server (local-fake remote) with OpenAI 

In [ ]:
import requests
from openai import OpenAI
import os 
from dotenv import load_dotenv
load_dotenv()

client_id_prod=os.getenv('client_id_prod')
client_secret_prod=os.getenv('client_secret_prod')


def get_token():
    token_url = 'https://oauth.piste.gouv.fr/api/oauth/token'
    #inject cred 
    token_data = {
    'grant_type': 'client_credentials',
    'client_id': client_id_prod,
    'client_secret': client_secret_prod,
    'scope': 'openid'}
    response = requests.post(token_url, data=token_data)
    response.raise_for_status()  # vérif  erreurs
    # récup  jeton
    token_info = response.json()
    access_token = token_info['access_token']
    return access_token


token=get_token()
github_barer=os.getenv('github_barer')
resp = OpenAI().responses.create(
    #model="gpt-5-nano-2025-08-07",
    model='gpt-5-mini-2025-08-07',
    tools=[
        {
            'type': 'mcp',
            'server_label': 'fetch',
            "server_url": "https://remote.mcpservers.org/fetch/mcp",
            "require_approval": "never"
        },
        {
            "type": "mcp",
            "server_label": "github",
            "server_url": "https://api.githubcopilot.com/mcp/",
            "headers": {
                "Authorization": f"Bearer {github_barer}",  
                "User-Agent": "myapp/1.0",
            },
            "require_approval": "never",
            
        },
        {
            'type': 'mcp',
            'server_label': 'OB_remote_server',
            "server_url": "https://bf5ef7873371.ngrok-free.app/sse",
            "require_approval": "never"
        }
        ],
#input="Montre-moi tous les outils MCP disponibles sur GitHub"
input="Sans appeler d'outil, dresse un inventaire des outils MCP que tu vois"
#input=f"récupère et compare les versions précédente et en vigueur du texte LEGIARTI000041577698 et trouve moi le numéro celex de la directive a l origine de la modification tu pourras utiliser le token suivant {token}  pour récupérer les infos "

)

print("------ Statistics ------")
print(f"Input tokens : {resp.usage.input_tokens}" )
print(f"Output tokens : {resp.usage.output_tokens}" )
#print(f"cached_tokens : {resp.usage.input_tokens_details.cached_tokens}")
print(f"Total tokens : {resp.usage.total_tokens}" )
print("")
print("")
print("------------------  Chatbot reponse  ------------------")
print(resp.output_text)

APIStatusError: Error code: 424 - {'error': {'message': "Error retrieving tool list from MCP server: 'OB_remote_server'. Http status code: 404 (Not Found)", 'type': 'external_connector_error', 'param': 'tools', 'code': 'http_error'}}